In [1]:
%load_ext autoreload
%autoreload 2

Install targen from pip

```bash 
pip install targen
```

In [2]:
from sklearn.datasets import make_classification

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from targen.data import target

### Get random gaussian data

In [3]:
n_features = 18

In [4]:
X, dummy = make_classification( n_samples=10000,
    n_features=n_features)

There are 20 features in the model, lets add some naming convention

- make sure that the first 12 feature are known and the last 8 are hidden
- Features that are predictive of the target will contain in p_y
- Features that are use for the accept/reject biasn will contain p_a


In [5]:
# def rename_columns(col,ix):
#     out = col
    
#     if ix <5:
#         out+='_p_y'
#     if 3<=ix<7:
#         out+='_p_a'
#     return out

known_cols =  [f'known_col_{ix}' for ix in range(int(n_features/2))]
if n_features %2==0:
    hidden_cols = [f'hidden_col_{ix}' for ix in range(int(n_features/2))]
    
else:
    hidden_cols = [f'hidden_col_{ix}' for ix in range(int(n_features/2)+1)]
columns = known_cols+hidden_cols

# known_cols = [rename_columns(col, ix) for ix,col in enumerate(known_cols)]
# hidden_cols = [rename_columns(col, ix) for ix,col in enumerate(hidden_cols)]

# columns = known_cols+hidden_cols

In [6]:
columns

['known_col_0',
 'known_col_1',
 'known_col_2',
 'known_col_3',
 'known_col_4',
 'known_col_5',
 'known_col_6',
 'known_col_7',
 'known_col_8',
 'hidden_col_0',
 'hidden_col_1',
 'hidden_col_2',
 'hidden_col_3',
 'hidden_col_4',
 'hidden_col_5',
 'hidden_col_6',
 'hidden_col_7',
 'hidden_col_8']

Map it to the dataframe

In [7]:
data = pd.DataFrame(X, columns = columns)

In [8]:
data

,known_col_0,known_col_1,known_col_2,known_col_3,known_col_4,known_col_5,known_col_6,known_col_7,known_col_8,hidden_col_0,hidden_col_1,hidden_col_2,hidden_col_3,hidden_col_4,hidden_col_5,hidden_col_6,hidden_col_7,hidden_col_8
0,-1.514834,-0.691054,-0.701259,-1.799825,0.503310,1.447505,-1.171768,0.192840,-0.129436,0.001499,0.575373,-0.146524,1.472080,-1.215229,0.681630,-1.296196,0.056276,-0.945076
1,1.225041,-0.762310,-0.279633,0.025562,0.757736,-0.688201,0.992133,1.113317,1.083114,-0.127719,1.703058,-1.106039,0.189622,-0.209269,-0.385268,0.962161,-0.474822,-0.154486
2,2.139550,0.905680,-0.086653,1.284909,1.679664,0.782860,-1.010833,0.724997,-0.287602,0.599644,0.746692,-1.073889,-1.325901,-0.518715,-0.789139,0.713595,0.097533,0.373812
3,1.091473,-0.327481,0.945175,-2.162692,-1.110672,-0.989450,1.183022,-0.032436,-0.638434,0.231871,-1.067052,-0.959236,-0.879711,-2.292089,-0.346814,-0.581400,0.572650,-0.117976
4,-1.514658,0.933908,-0.387462,1.022761,0.149401,-1.380624,-0.002899,-0.390368,-0.492954,0.526957,-0.029018,0.998902,-0.207346,-1.532102,0.526311,-0.821466,-1.328247,-0.085567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-2.109463,-0.353046,-0.028312,1.406722,-1.278636,0.439233,-0.647358,0.791608,0.217126,-0.035700,-0.742330,0.330811,-0.890600,-0.492601,0.876706,-0.298169,-0.648194,-0.914756
9996,0.618890,0.304681,-0.184782,0.343808,0.466266,-1.037166,0.326029,-0.162732,0.235269,-0.230822,-0.168607,0.849749,-2.355864,-0.573501,-0.385537,0.080061,-0.236815,0.978765
9997,0.098591,-0.108223,-0.500086,-1.342400,-0.450464,-1.063440,1.537264,-0.610378,-0.584345,1.060657,0.354948,-1.803055,-0.158282,-1.758656,0.201301,1.424341,-0.142520,-1.298478
9998,2.170572,-0.213255,1.053417,0.047275,0.045253,0.172979,0.434827,0.559104,0.385343,-0.845817,-1.829475,-1.036351,-0.903235,-0.391823,-0.807779,-0.685246,-0.855213,0.419080


### Add some correlation between variables

This function will be improved

In [9]:
def add_correlation_between_columns(df,col_1, col_2, theta_pi_2=1):

    theta = theta_pi_2 *0.5*np.pi
    
    data_out = df.copy()
    data_out[col_1] = -1*np.cos(theta)* df[col_1] + np.sin(theta* df[col_2])
    data_out[col_2] = np.cos(theta)* df[col_1] + np.sin(theta* df[col_2])
    
    return data_out
    

`theta_pi_2` is the angle of rotation (expressed in pi/2).<br>

When setting it to 1, you will add 100% correaltion between the variables.


In [10]:
new_data = (
    data
    .pipe(add_correlation_between_columns,col_1 = 'known_col_0', col_2 = 'known_col_1', theta_pi_2 =1)
    .pipe(add_correlation_between_columns,col_1 = 'known_col_2', col_2 = 'known_col_3', theta_pi_2 =-0.9)
    .pipe(add_correlation_between_columns,col_1 = 'known_col_2', col_2 = 'hidden_col_0', theta_pi_2 =-0.8)
    .pipe(add_correlation_between_columns,col_1 = 'hidden_col_1', col_2 = 'known_col_1', theta_pi_2 =0.4)
              
)

Feel free to add more correlations between variables

In [ ]:
new_data.corr()[["known_col_0","known_col_1","known_col_2","known_col_3","hidden_col_0","hidden_col_1"]]

# Define all expressions in one go and get the targets

Quick way to get your y target.<br>

Define the expression dictionary as in the example below.

Allowed keys are

- linear: expects a value being a string with an expression
- non_linear: expects a value being a string with an expression
- interaction: expects a value being a string with an expression
- conditional: expects a dictionary, where:<br>
    - keys: are the conditions
    - values are tuples:
        - the first element is the expression or a value to assign to in case when the conditions is true
        - the second element is the expression or a value to assign to in case when the conditions is false

- uniform noise: expects a dictionary with kwargs to apply, options are
    - weight
    - min (for scaling)
    - max (for scaling)

    
- gaussian noise: expects a dictionary with kwargs to apply, options are
    - weight
    - mu_gaus
    - sigma_gaus



# Get the relationships
- example: make columns 0,1,2,3,4 predictive for y
- make columns 3,4,5,6 predictive for acceptance
- columns 7 and 8 are dummy columns

### Generate X-y relationship (feature-target)



In [ ]:
expressions_x_y = {
   "linear":'-0.5*known_col_0 + 2.*known_col_1 + 0.7*known_col_4 -0.4*hidden_col_0',
    'non_linear': '-0.7*known_col_3**1.5 + 0.2*sin(known_col_1)+ 0.9*log(hidden_col_4) -0.1*hidden_col_2**2',
    'interaction': '0.05*known_col_3*known_col_4 -0.1*(known_col_4/hidden_col_1)',
    'uniform_noise': {
        'weight':0.5
    },
}

### Generate accept-reject relationship

In [ ]:
expressions_accept_reject = {
   "linear":'-14*known_col_4 + 2.*known_col_5 + 0.9*hidden_col_6 -2.1*known_col_3',
    'non_linear': '-0.9*known_col_3**0.5 + 0.2*sin(known_col_5) -2.5*log(hidden_col_5)-0.05*hidden_col_6**2.5',
#     'interaction': '0.01*known_col_3*known_col_4 -0.05*(hidden_col_5/hidden_col_6)',
    'uniform_noise': {
        'weight':0.1
    },
}

## Steps to try:

- Add more complex relationship between Xy( accept-rejects)
- Add correlations (have a few features in the KGB model that are not used directly in Accept-reject, but are correlated with some features in accept reject)
- Tune the predictability on the KGB sample (make the model more predictable there, but not on the reject sample...)
- ...

### Apply `targen` and get the new dataset

In [ ]:
X_y = target.get_target_and_contributions(data, expressions=expressions_x_y, imbalance = 0.05, drop_features=False)

In [ ]:
X_y['y'].value_counts()

In [ ]:
X_acc_reject = target.get_target_and_contributions(data, expressions=expressions_accept_reject, imbalance = 0.7, drop_features=False)

In [ ]:
X_acc_reject = X_acc_reject.rename(columns={'y':'is_accepted'})

In [ ]:
X_acc_reject['is_accepted'].value_counts()

# Put all the data together

In [ ]:
dataset = pd.concat([X_y,X_acc_reject['is_accepted']], axis=1).drop([col for col in X_y.columns if col.startswith("score") ], axis=1)

In [ ]:
dataset.head()

### Check the default rate per samples (kgb vs rejected)

In [ ]:
kgb = dataset[dataset["is_accepted"]==1]
rejected = dataset[dataset["is_accepted"]==0]

In [ ]:
kgb['y'].value_counts(normalize=True)

In [ ]:
rejected['y'].value_counts(normalize=True)

### Get the modelling dataset

Assume you can use only the known columns fot the evaluation of your data

In [ ]:
model_ds = dataset[[col for col in dataset.columns if col.startswith("known")]+["y","is_accepted"]]
model_ds

# From here onwards build the Reject Inference cases

An option could be to save the generated samples to a csv and then use a different notebook to run the analysis